# Create table from csv files (basic)
Lee una serie de ficheros csv con los datos que se desean importar a nuevas tablas (una tabla para cada csv)<br>
__Uso__
1. Asignar valor a los parámetros 
1. Ejecutar la función tables_columns_2_rename

In [1]:
import csv
import glob
import os
from os.path import join


## Functions

In [9]:
def tables_columns_2_rename(files: [str], to_screen: bool=False) -> [[str], [str]]:
    """
    Escribe el nombre de todas las tablas en files para ser renombradas como nombre de las tablas
    Extrae la cabecera de cada fichero csv y escribe en un fichero todas la cabeceras para que sean
    renombradas como las columnas de las tablas
    ----
    files: lista de paths csv a tratar
    to_screen: if True print intermediate results
    ---
    return [name_tables, name_columns]
    name_tables: nombres de los ficheros sin path
    name_columns: nombres de las columnas
    """
    name_tables = []
    name_columns = []

    for file in files:
        with open(file, mode='r', encoding='utf-8') as fi:
            rfi = csv.reader(fi, delimiter=',')
            table_name, _ = os.path.basename(file).split('.')
            name_tables.append(table_name)
            for ir, row in enumerate(rfi):
                if to_screen:
                    print(table_name)
                    print(row)
                    print('')
                for col1 in row:
                    if col1 not in name_columns:
                        name_columns.append(col1)
                break

    return name_tables, name_columns


def names_get(file: str) -> dict:
    """
    file: path/file con los nombres (tablas o columnas según file) en los ficheros
        csv y los que se van a utilizar en las tablas
    ----
    return:
    dict en el que key es el nombre en el fichero csv y atrib es nombre a utilizar
    """
    with open(file, 'r', encoding='utf-8') as fi:
        rfi = csv.reader(fi, delimiter=',')
        names = [(row[0], row[1])  for row in rfi]
    return dict(names)
        

def sets_get():
    """
    valores de los parámetros para escribir en el fichero sql donde se crean las tablas
    """
    sets = \
"""
-- PostgreSQL database dump

SET statement_timeout = 0;
SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;
SET xmloption = content;
SET client_min_messages = warning;
SET row_security = off;

SET default_tablespace = '';

SET default_table_access_method = heap;

"""    
    return sets


def tables_columns_translate(files: [str], to_screen: bool=False) -> [[str], [str]]:
    """
    
    ----
    files: lista de paths csv a tratar
    to_screen: if True print intermediate results
    ---
    return [name_tables, name_columns]
    name_tables: nombres de los ficheros sin path
    name_columns: nombres de las columnas
    """
    name_tables = []
    name_columns = []

    for file in files:
        with open(file, mode='r', encoding='utf-8') as fi:
            rfi = csv.reader(fi, delimiter=',')
            table_name, _ = os.path.basename(file).split('.')
            name_tables.append(table_name)
            for ir, row in enumerate(rfi):
                if to_screen:
                    print(table_name)
                    print(row)
                    print('')
                for col1 in row:
                    if col1 not in name_columns:
                        name_columns.append(col1)
                break

    return name_tables, name_columns

def create_table_sql(table_name, col_names, schema = ''):
    cols = [col1 + ' varchar' for col1 in col_names]
    cols = ',\n'.join(cols)
    if len(schema) > 0:
        schema = schema + '.'
    comment = f"comment on table {schema}{table_name} is 'CHJ Plan Hidrológico 21-27 {table_name}';\n"
    comment = comment.replace('_', ' ')
    
    comment_columns = [f"comment on columnn {schema}{table_name}.{col1} is '{col1}';" for col1 in col_names]
    comment_columns = '\n'.join(comment_columns)
    
    command = f"""
create table if not exists {schema}{table_name} (
{cols}    
);

{comment}
{comment_columns}
    """
    return command

## Set parameter values

In [4]:
# directorio y expression to get csv files
drt = r'E:\PPHH\chj_22_27\tablas'
mask = '*.csv'

# file_name_tables: nombre del fichero donde se grabarán el nombre d elas tablas
file_name_tables = join(drt, '01_table_names.txt')
# file_name_columns: nombre del fichero donde se grabarán el nombre de las columnas 
file_name_columns = join(drt, '02_colum_names.txt')

# fo_name: nombre del fichero donde se grabarán los comandos parara creación de las tablas
fo_name = join(drt, '03_create_tables.sql')

# schema: nombre del esquema de la DB donde se crearán las tablas
schema = 'tmp'


## Function 1. Write names of files and columns
Después de crearse el fichero deben darse los nuevos nobres manualmente en el fichero

In [6]:
files = glob.glob(join(drt, mask))

tables, columns = tables_columns_2_rename(files)

with open(file_name_tables, 'w', encoding='utf-8') as fo:
    fo.write('name,new_name\n')
    for t1 in tables:
        fo.write(f'{t1},\n')

with open(file_name_columns, 'w', newline='', encoding='utf-8') as fo:
    fo.write('name,new_name\n')
    for c1 in columns:
        fo.write(f'{c1},\n')


## Function 2. Write file create_tables.sql
Después de asignar los nombres de las tablas y las columnas a partir de los 2 ficheros creados en la celda anterior, se crea un fichero sql en el que se crean las tablas

In [11]:
table_names = names_get(join(drt, '01_table_names_done.txt'))
column_names = names_get(join(drt, '02_colum_names_done.txt'))



In [14]:
column_names

{'name': 'new_name',
 'Código Masa subterránea': 'msbt_id',
 'Masa subterránea': 'msbt_name',
 'Recurso renovable': 'recurso_renov',
 'Restricciones ambientales': 'retrict_amb',
 'Recurso disponible': 'recurso_disp',
 'Bombeo total': 'bombeo_tot',
 'Derecho total': 'derecho_tot',
 'Derecho sección B': 'derecho_sec_b',
 'Índice de explotación con usos K': 'ie_con_usos_k',
 'Índice de explotación con derechos': 'ie_con_derechos',
 'Código Masa': 'msbt_id',
 'Nombre Masa': 'msbt_name',
 'Facies hidroquímica': 'facies_hq',
 'Motivo Mal Estado*': 'causa_mal_estado',
 'Código masa superficial asociada en estado peor que bueno': 'msf_id_mal_estado',
 'Nombre masa superficial asociada en estado peor que bueno': 'msf_name_mal_estado',
 'Cumple caudal ecológico mínimo': 'cumple_q_ecolog_min',
 'Índice de explotación': 'ie',
 'Evaluación tendencia piezométrica': 'tendencia_pz',
 'Estado test 2 masas superficiales asociadas': 'estado_t2_msf',
 'EDA dañado o en riesgo': 'eda_damaged_risk',
 'Descen

In [ ]:
with open(fo_name, 'w') as fo:
    fo.write(sets_get())
    for file in files:
        with open(file, mode='r', encoding='utf-8') as fi:
            rfi = csv.reader(fi, delimiter=',')
            table_name, _ = os.path.basename(file).split('.')
            for ir, row in enumerate(rfi):
                print(table_name)
                print(row)
                print('')
                sql_command = create_table_sql(table_name, row, schema = schema)
                break
            fo.write(f'{sql_command}\n\n')